## Calculate attributes per subbasin
Takes prepared geospatial data and computes various attributes.

In [1]:
import geopandas as gpd
import pandas as pd
import sys
from pathlib import Path
sys.path.append(str(Path().absolute().parent))
from python_cs_functions import config as cs, attributes as csa
from python_cs_functions.delineate import prepare_delineation_outputs

#### DEV only

In [2]:
import geopandas as gpd
import glob
import numpy as np 
import os
import xarray as xr
from rasterstats import zonal_stats
from python_cs_functions.attributes import read_scale_and_offset, zonal_out_none2nan, subset_dataset_to_max_full_years, process_monthly_means_to_lists
from python_cs_functions.attributes import find_climate_seasonality_rdrs, circmean_group, circstd_group, calculate_temp_prcp_stats, find_durations, get_season
from python_cs_functions.attributes import create_mean_daily_max_series, get_open_water_stats, get_categorical_dict, check_scale_and_offset
from scipy.stats import circmean, circstd, skew, kurtosis

### Config handling

In [2]:
# Specify where the config file can be found
config_file = '../0_config/config.txt'

In [12]:
# Get the required info from the config file
data_path            = cs.read_from_config(config_file,'data_path')

# CAMELS-spat metadata
cs_meta_path = cs.read_from_config(config_file,'cs_basin_path')
cs_meta_name = cs.read_from_config(config_file,'cs_meta_name')
cs_unusable_name = cs.read_from_config(config_file,'cs_unusable_name')

# Basin folder
cs_basin_folder = cs.read_from_config(config_file, 'cs_basin_path')
basins_path = Path(data_path) / cs_basin_folder

# Get the temporary data folder
cs_temp_folder = cs.read_from_config(config_file, 'temp_path')
temp_path = Path(cs_temp_folder)
temp_path.mkdir(exist_ok=True, parents=True)

# Get the attribute folder
att_folder = cs.read_from_config(config_file, 'att_path')
att_path = basins_path / att_folder / 'distributed'
att_path.mkdir(parents=True, exist_ok=True)

# Get the CRS we want to do area calculations in (possibly helpful for GLHYMPS and HydroLAKES)
ea_crs = cs.read_from_config(config_file, 'equal_area_crs')

### Data loading

In [4]:
# CAMELS-spat metadata file
cs_meta_path = Path(data_path) / cs_meta_path
cs_meta = pd.read_csv(cs_meta_path / cs_meta_name)

In [5]:
# Open list of unusable stations; Enforce reading IDs as string to keep leading 0's
cs_unusable = pd.read_csv(cs_meta_path / cs_unusable_name, dtype={'Station_id': object})

### Processing

In [6]:
debug_message = f'\n!!! CHECK DEBUGGING STATUS: \n- Testing 1 file \n- Testing 1 basin'

In [7]:
# Removed 'hydrology', because we don't have gauges for every subbasin
data_subfolders = ['rdrs', 'worldclim', 'lai', 'forest_height', 'glclu2019', 'modis_land', 'lgrip30', 'merit', 'hydrolakes', 'pelletier', 'soilgrids', 'glhymps']

### DEV

In [22]:
# Set up a relatively simple test case (not too big)
ix = 9
row = cs_meta.iloc[ix]
print(f"{row['Country']}_{row['Station_id']}")

CAN_01AK006


In [23]:
# Get the paths
basin_id, shp_lump_path, shp_dist_path, _, _ = prepare_delineation_outputs(cs_meta, ix, basins_path)
geo_folder = basins_path / 'basin_data' / basin_id / 'geospatial'
met_folder = basins_path / 'basin_data' / basin_id / 'forcing'
hyd_folder = basins_path / 'basin_data' / basin_id / 'observations'

In [24]:
# Define the shapefiles
shp = str(shp_dist_path).format('basin') # because zonalstats wants a file path, not a geodataframe
riv = str(shp_dist_path).format('river') # For topographic attributes

Test the different data products

#### Fix the distributed case for geotiffs
Trial with forest_height (general case), soilgrids (special averaging function), glclu (categorical outcomes)

In [71]:
# Load the shapefile to get the sub-basin order for geotiffs and shapefiles
gdf = gpd.read_file(shp)
l_comids_geo = gdf['COMID'].to_list() # Store the polygon IDs into a list, we'll later use this as columns in the attribute dataframes

In [93]:
# Initialize storage lists for geotiff and shapefile attributes
l_values_geo = [[] for _ in range(len(l_comids_geo))] # Initialize an empty list where we'll store this basin's attributes - nested lists for each subbasin
l_index_geo = [] # Initialize an empty list where we'll store the attribute descriptions - this will be our dataframe index

# geotiffs
dataset = 'forest_height'
if dataset == 'forest_height':
    l_values_geo, l_index_geo = attributes_from_forest_height(geo_folder, dataset, shp, l_values_geo, l_index_geo, case='distributed')

dataset = 'soilgrids'
if dataset == 'soilgrids':
    l_values_geo, l_index_geo = csa.attributes_from_soilgrids(geo_folder, dataset, shp, l_values_geo, l_index_geo, case='distributed')

dataset = 'glclu2019'
if dataset == 'glclu2019':
    l_values_geo, l_index_geo = csa.attributes_from_glclu2019(geo_folder, dataset, shp, l_values_geo, l_index_geo, case='distributed')

dataset = 'merit'
l_values_geo, l_index_geo, l_comids_merit = attributes_from_merit(geo_folder, dataset, shp, riv, row, l_values_geo, l_index_geo, equal_area_crs=ea_crs, case='distributed')
assert (l_comids_geo == l_comids_merit).all(), f"mismatch between COMID orders determined before and inside attributes_from_merit()"

TypeError: attributes_from_soilgrids() got an unexpected keyword argument 'case'

In [91]:
# create the geospatial attribute dataframe
att_df_geo = pd.DataFrame(data = l_values_geo, index = l_comids_geo).transpose()
multi_index = pd.MultiIndex.from_tuples(l_index_geo, names=['Category', 'Attribute', 'Unit', 'Source'])
att_df_geo.index = multi_index

In [87]:
def update_values_list(l_values, stats, zonal_out, scale, offset, case='lumped'):

    # Update scale and offset to usable values
    if scale is None: scale = 1 # If scale is undefined that means we simply multiply by 1
    if offset is None: offset = 0 # Undefined offset > add 0

    # Deal with the occassional None that we get when raster data input to zonal-stats is missing
    # This fixes 9 occurrences of missing data in the soilgrids maps
    zonal_out = zonal_out_none2nan(zonal_out)
    
    # We loop through the calculated stats in a pre-determined order:
    # 1. min
    # 2. mean
    # 3. max
    # 4. stdev
    # 5. ..
    if case == 'lumped':
        if 'min' in stats:  l_values.append(zonal_out[0]['min']  * scale + offset)
        if 'mean' in stats: l_values.append(zonal_out[0]['mean'] * scale + offset)
        if 'harmonic_mean'  in stats: l_values.append(zonal_out[0]['harmonic_mean'] * scale + offset) # only here for soilgrids conductivity, in which case we don't have 'mean'
        if 'max' in stats:  l_values.append(zonal_out[0]['max']  * scale + offset)
        if 'std' in stats:  l_values.append(zonal_out[0]['std']  * scale + offset)

    # distributed case, multiple polygons
    elif case == 'distributed':
        
        # confirm that l_values has as many nested lists as we have zonal stats outputs
        num_nested_lists = sum(1 for item in l_values if isinstance(item, list))
        assert num_nested_lists == len(zonal_out), f"zonal_out length does not match expected list length {num_nested_lists}. zonal_out: {zonal_out}"
        
        # now loop over the zonal outputs and append to relevant lists
        for i in range(0,num_nested_lists):
            if 'min' in stats:  l_values[i].append(zonal_out[i]['min']  * scale + offset)
            if 'mean' in stats: l_values[i].append(zonal_out[i]['mean'] * scale + offset)
            if 'harmonic_mean'  in stats: l_values[i].append(zonal_out[i]['harmonic_mean'] * scale + offset) # only here for soilgrids conductivity, in which case we don't have 'mean'
            if 'max' in stats:  l_values[i].append(zonal_out[i]['max']  * scale + offset)
            if 'std' in stats:  l_values[i].append(zonal_out[i]['std']  * scale + offset)

    return l_values

In [13]:
def harmonic_mean(x):
    return np.ma.count(x) / (1/x).sum()

In [14]:
def attributes_from_soilgrids(geo_folder, dataset, shp_str, l_values, l_index):

    '''Calculates attributes from SOILGRIDS maps'''

    # File specifiction
    sub_folders = ['bdod',     'cfvo',       'clay',    'sand',    'silt',    'soc',      'porosity']
    units       = ['cg cm^-3', 'cm^3 dm^-3', 'g kg^-1', 'g kg^-1', 'g kg^-1', 'dg kg^-1', '-']
    depths = ['0-5cm', '5-15cm', '15-30cm', '30-60cm', '60-100cm', '100-200cm']
    fields = ['mean', 'uncertainty']
    stats = ['mean', 'min', 'max', 'std']
    
    # Loop over the files and calculate stats
    for sub_folder, unit in zip(sub_folders, units):
        for depth in depths:
            for field in fields:
                tif = str(geo_folder / dataset / 'raw' / f'{sub_folder}' / f'{sub_folder}_{depth}_{field}.tif')
                if not os.path.exists(tif): continue # porosity has no uncertainty field
                zonal_out = zonal_stats(shp_str, tif, stats=stats)
                scale,offset = read_scale_and_offset(tif)
                l_values = update_values_list(l_values, stats, zonal_out, scale, offset)
                l_index += [('Soil', f'{sub_folder}_{depth}_{field}_min',  f'{unit}', 'SOILGRIDS'),
                            ('Soil', f'{sub_folder}_{depth}_{field}_mean', f'{unit}', 'SOILGRIDS'),
                            ('Soil', f'{sub_folder}_{depth}_{field}_max',  f'{unit}', 'SOILGRIDS'),
                            ('Soil', f'{sub_folder}_{depth}_{field}_std',  f'{unit}', 'SOILGRIDS')]
           
    # --- Specifc processing for the conductivity fields
    # For conductivity we want to have a harmonic mean because that should be more
    # representative as a spatial average. rasterstats doesn't have that, so we 
    # need a custom function. We're splitting out the processing from the rest for
    # clarity. Could also have done this with a bunch of if-statements, but this 
    # seems cleaner, also because conductivity doesn't have uncertainty maps.
    
    # Process the data fields
    sub_folder = 'conductivity'
    unit       = 'cm hr^-1'
    depths = ['0-5cm', '5-15cm', '15-30cm', '30-60cm', '60-100cm', '100-200cm']
    field  = 'mean' # no uncertainty maps for these
    stats = ['min', 'max', 'std']

    for depth in depths:
        tif = str(geo_folder / dataset / 'raw' / f'{sub_folder}' / f'{sub_folder}_{depth}_{field}.tif')
        if not os.path.exists(tif): continue # porosity has no uncertainty field
        zonal_out = zonal_stats(shp_str, tif, stats=stats, add_stats={'harmonic_mean': harmonic_mean})
        scale,offset = read_scale_and_offset(tif)
        l_values = update_values_list(l_values, ['min', 'max', 'std', 'harmonic_mean'], zonal_out, scale, offset)
        l_index += [('Soil', f'{sub_folder}_{depth}_{field}_min',  f'{unit}', 'SOILGRIDS'),
                    ('Soil', f'{sub_folder}_{depth}_{field}_harmonic_mean', f'{unit}', 'SOILGRIDS'),
                    ('Soil', f'{sub_folder}_{depth}_{field}_max',  f'{unit}', 'SOILGRIDS'),
                    ('Soil', f'{sub_folder}_{depth}_{field}_std',  f'{unit}', 'SOILGRIDS')]

    return l_values, l_index

In [89]:
def attributes_from_forest_height(geo_folder, dataset, shp_str, l_values, index, case='lumped'):

    '''Calculates mean, min, max and stdv for forest height 2000 and 2020 tifs'''

    # Year 2000 min, mean, max, stdev
    tif = str( geo_folder / dataset / 'raw' / 'forest_height_2000.tif' )
    stats = ['mean', 'min', 'max', 'std']
    zonal_out = zonal_stats(shp_str, tif, stats=stats)
    scale,offset = read_scale_and_offset(tif)
    l_values = update_values_list(l_values, stats, zonal_out, scale, offset, case=case)
    index += [('Land cover', 'forest_height_2000_min',   'm', 'GLCLUC 2000-2020'),
              ('Land cover', 'forest_height_2000_mean',  'm', 'GLCLUC 2000-2020'),
              ('Land cover', 'forest_height_2000_max',   'm', 'GLCLUC 2000-2020'),
              ('Land cover', 'forest_height_2000_std',   'm', 'GLCLUC 2000-2020')]

    # Year 2020 mean, stdev
    tif = geo_folder / dataset / 'raw' / 'forest_height_2020.tif'
    stats = ['mean', 'min', 'max', 'std']
    zonal_out = zonal_stats(shp_str, tif, stats=stats)
    scale,offset = read_scale_and_offset(tif)
    l_values = update_values_list(l_values, stats, zonal_out, scale, offset, case=case)
    index += [('Land cover', 'forest_height_2020_min',   'm', 'GLCLUC 2000-2020'),
              ('Land cover', 'forest_height_2020_mean',  'm', 'GLCLUC 2000-2020'),
              ('Land cover', 'forest_height_2020_max',   'm', 'GLCLUC 2000-2020'),
              ('Land cover', 'forest_height_2020_std',   'm', 'GLCLUC 2000-2020')]

    return l_values, index

In [42]:
def attributes_from_glclu2019(geo_folder, dataset, shp_str, l_values, l_index):

    '''Calculates percentage occurrence of all classes in GLCLU2019 map'''

    tif = geo_folder / dataset / 'raw' / 'glclu2019_map.tif'
    zonal_out = zonal_stats(shp_str, tif, categorical=True)
    check_scale_and_offset(tif)
    l_values,l_index = update_values_list_with_categorical(l_values, l_index, zonal_out, 'GLCLU 2019', prefix='lc1_')
    return l_values, l_index

In [64]:
def update_values_list_with_categorical(l_values, l_index, zonal_out, source, prefix='', case='lumped'):
    '''Maps a zonal histogram of categorical classes onto descriptions and adds to lists'''

    # Get the category definitions
    cat_dict = get_categorical_dict(source)    

    # Separately handle lumped and distributed cases
    if case == 'lumped':
    
        # Find the total number of classified pixels
        total_pixels = 0
        for land_id,count in zonal_out[0].items():
            total_pixels += count
        
        # Loop over all categories and see what we have in this catchment
        for land_id,text in cat_dict.items():
            land_prct = 0
            if land_id in zonal_out[0].keys():
                land_prct = zonal_out[0][land_id] / total_pixels
            l_values.append(land_prct)
            l_index.append(('Land cover', f'{prefix}{text}_fraction', '-', f'{source}'))

    # distributed case, multiple polygons
    elif case == 'distributed': 

        # confirm that l_values has as many nested lists as we have zonal stats outputs
        num_nested_lists = sum(1 for item in l_values if isinstance(item, list))
        assert num_nested_lists == len(zonal_out), f"zonal_out length does not match expected list length {num_nested_lists}. zonal_out: {zonal_out}"

        # now loop over the zonal outputs and append to relevant lists
        for i in range(0,num_nested_lists):

            # Find the total number of classified pixels
            total_pixels = 0
            for land_id,count in zonal_out[i].items():
                total_pixels += count
            
            # Loop over all categories and see what we have in this catchment
            tmp_index = [] # we need this so the index resets on each subbasin iteration, and we need that because we only need the index once
            for land_id,text in cat_dict.items():
                land_prct = 0
                if land_id in zonal_out[i].keys():
                    land_prct = zonal_out[i][land_id] / total_pixels
                l_values[i].append(land_prct)
                tmp_index.append(('Land cover', f'{prefix}{text}_fraction', '-', f'{source}'))

        # Add the index values only once
        for item in tmp_index:
            l_index.append(item)

    return l_values,l_index

#### Fix the WorldClim distributed case

In [47]:
# Data storage
l_comids_geo = gdf['COMID'].to_list() # Store the polygon IDs into a list, we'll later use this as columns in the attribute dataframes
l_values_geo = [[] for _ in range(len(l_comids_geo))] # Initialize an empty list where we'll store this basin's attributes - nested lists for each subbasin
l_index_geo = [] # Initialize an empty list where we'll store the attribute descriptions - this will be our dataframe index

dataset = 'worldclim'
if dataset == 'worldclim':
    csa.oudin_pet_from_worldclim(geo_folder, dataset) # Get an extra PET estimate to sanity check RDRS outcomes
    csa.aridity_and_fraction_snow_from_worldclim(geo_folder, dataset) # Get monthly aridity and fraction snow maps
    l_values, l_index = attributes_from_worldclim(geo_folder, dataset, shp, l_values_geo, l_index_geo)

# - geotiffs
att_df_geo = pd.DataFrame(data = l_values_geo, index = l_comids_geo).transpose()
multi_index = pd.MultiIndex.from_tuples(l_index_geo, names=['Category', 'Attribute', 'Unit', 'Source'])
att_df_geo.index = multi_index
att_df_geo

72048321     72049179  \
Category Attribute               Unit Source                                
Climate  prec_mean               mm   WorldClim  1269.745763  1348.750000   
         prec_std                mm   WorldClim    51.918970    34.964565   
         pet_mean                mm   WorldClim   256.894862   247.144816   
         pet_std                 mm   WorldClim     4.371759     4.058496   
         tavg_mean               C    WorldClim     5.410735     4.851562   
...                                                      ...          ...   
         fracsnow2_std_month_10  -    WorldClim     0.000000     0.000000   
         fracsnow2_mean_month_11 -    WorldClim     0.000000     0.000000   
         fracsnow2_std_month_11  -    WorldClim     0.000000     0.000000   
         fracsnow2_mean_month_12 -    WorldClim     1.000000     1.000000   
         fracsnow2_std_month_12  -    WorldClim     0.000000     0.000000   

                                                    72049188  
Category Attribute               Unit Source                  
Climate  prec_mean               mm   WorldClim  1359.958333  
         prec_std                mm   WorldClim    29.905377  
         pet_mean                mm   WorldClim   248.320435  
         pet_std                 mm   WorldClim     3.083181  
         tavg_mean               C    WorldClim     4.921528  
...                                                      ...  
         fracsnow2_std_month_10  -    WorldClim     0.000000  
         fracsnow2_mean_month_11 -    WorldClim     0.000000  
         fracsnow2_std_month_11  -    WorldClim     0.000000  
         fracsnow2_mean_month_12 -    WorldClim     1.000000  
         fracsnow2_std_month_12  -    WorldClim     0.000000  

[252 rows x 3 columns]

In [46]:
import glob
import os
from rasterstats import zonal_stats
from python_cs_functions.attributes import read_scale_and_offset, update_values_list,zonal_stats_unit_conversion,create_annual_worldclim_map
from python_cs_functions.attributes import derive_annual_worldclim_aridity,derive_annual_worldclim_seasonality,derive_annual_worldclim_fracsnow

In [33]:
def attributes_from_worldclim(geo_folder, dataset, shp_str, l_values, l_index):

    '''Calculates mean and stdv for tifs of monthly WorldClim values'''

    # Define file locations
    # Units source: https://www.worldclim.org/data/worldclim21.html
    clim_folder = geo_folder / dataset / 'raw'
    sub_folders =      ['prec', 'srad',   'tavg', 'tmax', 'tmin', 'vapr', 'wind',   'pet', 'aridity2', 'fracsnow2'] # aridity and fractionsnow have subscript 2 to distinguish them from ERA5 attributes
    sub_folder_units = ['mm',   'W m^-2', 'C',    'C',    'C',    'kPa',  'm s^-1', 'mm',  '-',       '-']  # srad original: kJ m^-2 d^-1. Converted below

    # Get the annual values
    l_values,l_index = get_annual_worldclim_attributes(clim_folder, shp_str, 'prec', 'tavg', 'pet', 'snow2', l_values, l_index)

    # Loop over the files and calculate the stats
    for sub_folder, sub_folder_unit in zip(sub_folders, sub_folder_units):
        month_files = sorted( glob.glob(str(clim_folder / sub_folder / '*.tif')) )
        for month_file in month_files:
            month_file = clim_folder / sub_folder / month_file # Construct the full path, because listdir() gives only files
            stats = ['mean', 'std']
            zonal_out = zonal_stats(shp_str, month_file, stats=stats)
            
            scale, offset = read_scale_and_offset(month_file)
            if sub_folder == 'srad':
                zonal_out = zonal_stats_unit_conversion(zonal_out,stats,'srad', scale, offset)

            l_values = update_values_list(l_values, stats, zonal_out, scale, offset)
            
            month = os.path.basename(month_file).split('_')[3].split('.')[0]
            var = os.path.basename(month_file).split('_')[2]
            source = 'WorldClim'
            if var == 'pet': source = 'WorldClim (derived, Oudin et al., 2005)'
            l_index += [('Climate', f'{var}_mean_month_{month}', f'{sub_folder_unit}',  source),
                        ('Climate', f'{var}_std_month_{month}', f'{sub_folder_unit}', source)]

    return l_values, l_index

In [39]:
def get_annual_worldclim_attributes(clim_folder, shp_str, prec_folder, tavg_folder, pet_folder, snow_folder, l_values, l_index):

    '''Calculates annual WorldClim statistics'''

    # Create the output folder
    ann_folder = clim_folder / 'annual'
    ann_folder.mkdir(exist_ok=True, parents=True)

    # General settings
    stats = ['mean', 'std']

    # We need some special actions if we're dealing with the distributed case, so check that first
    case = 'lumped'
    if 'distributed' in shp_str:
        case = 'distributed'
        num_nested_lists = len(l_values)
    
    # --- P
    prec_files = sorted( glob.glob(str(clim_folder / prec_folder / '*.tif')) )
    annual_prec = create_annual_worldclim_map(prec_files, ann_folder, 'prec_sum.tif', 'prec')
    zonal_out = zonal_stats(shp_str, annual_prec, stats=stats)

    # Handle scale and offset
    scale,offset = read_scale_and_offset(annual_prec)
    if scale is None: scale = 1 # If scale is undefined that means we simply multiply by 1
    if offset is None: offset = 0 # Undefined offset > add 0

    # Update lists
    for stat in stats:
        if case == 'lumped':
            l_values.append(zonal_out[0][stat] * scale + offset)
        elif case == 'distributed':
            for i in range(0,num_nested_lists):
                l_values[i].append(zonal_out[i][stat] * scale + offset)
        l_index.append(('Climate',f'prec_{stat}','mm', 'WorldClim'))

    # --- PET
    pet_files = sorted( glob.glob(str(clim_folder / pet_folder / '*.tif')) )
    annual_pet = create_annual_worldclim_map(pet_files, ann_folder, 'pet_sum.tif', 'pet')
    zonal_out = zonal_stats(shp_str, annual_pet, stats=stats)
    
    scale,offset = read_scale_and_offset(annual_pet)
    if scale is None: scale = 1
    if offset is None: offset = 0
        
    for stat in stats:
        if case == 'lumped':
            l_values.append(zonal_out[0][stat] * scale + offset)
        elif case == 'distributed':
            for i in range(0,num_nested_lists):
                l_values[i].append(zonal_out[i][stat] * scale + offset)
        l_index.append(('Climate',f'pet_{stat}','mm', 'WorldClim'))
        
    # --- T
    tavg_files = sorted( glob.glob(str(clim_folder / tavg_folder / '*.tif')) )
    annual_tavg = create_annual_worldclim_map(tavg_files, ann_folder, 't_avg.tif', 'tavg')
    zonal_out = zonal_stats(shp_str, annual_tavg, stats=stats)
    
    scale,offset = read_scale_and_offset(annual_tavg)
    if scale is None: scale = 1
    if offset is None: offset = 0
        
    for stat in stats:
        if case == 'lumped':
            l_values.append(zonal_out[0][stat] * scale + offset)
        elif case == 'distributed':
            for i in range(0,num_nested_lists):
                l_values[i].append(zonal_out[i][stat] * scale + offset)
        l_index.append(('Climate',f'tavg_{stat}','C', 'WorldClim'))

    # --- Snow
    snow_files = sorted( glob.glob(str(clim_folder / snow_folder / '*.tif')) )
    annual_snow = create_annual_worldclim_map(snow_files, ann_folder, 'snow_sum.tif', 'snow')

    # --- Aridity
    annual_ari = derive_annual_worldclim_aridity(annual_prec, annual_pet, ann_folder, 'aridity.tif')
    zonal_out = zonal_stats(shp_str, annual_ari, stats=stats)
    
    scale,offset = read_scale_and_offset(annual_ari)
    if scale is None: scale = 1
    if offset is None: offset = 0
        
    for stat in stats:
        if case == 'lumped':
            l_values.append(zonal_out[0][stat] * scale + offset)
        elif case == 'distributed':
            for i in range(0,num_nested_lists):
                l_values[i].append(zonal_out[i][stat] * scale + offset)
        l_index.append(('Climate',f'aridity2_{stat}','-', 'WorldClim'))

    # --- Seasonality
    annual_seas = derive_annual_worldclim_seasonality(prec_files, tavg_files, ann_folder, 'seasonality.tif')
    zonal_out = zonal_stats(shp_str, annual_seas, stats=stats)
    
    scale,offset = read_scale_and_offset(annual_seas)
    if scale is None: scale = 1
    if offset is None: offset = 0
        
    for stat in stats:
        if case == 'lumped':
            l_values.append(zonal_out[0][stat] * scale + offset)
        elif case == 'distributed':
            for i in range(0,num_nested_lists):
                l_values[i].append(zonal_out[i][stat] * scale + offset)
        l_index.append(('Climate',f'seasonality2_{stat}','-', 'WorldClim'))
    
    # --- Snow fraction
    annual_fs = derive_annual_worldclim_fracsnow(annual_prec, annual_snow, ann_folder, 'fracsnow.tif')
    zonal_out = zonal_stats(shp_str, annual_fs, stats=stats)
    
    scale,offset = read_scale_and_offset(annual_fs)
    if scale is None: scale = 1
    if offset is None: offset = 0
        
    for stat in stats:
        if case == 'lumped':
            l_values.append(zonal_out[0][stat] * scale + offset)
        elif case == 'distributed':
            for i in range(0,num_nested_lists):
                l_values[i].append(zonal_out[i][stat] * scale + offset)
        l_index.append(('Climate',f'fracsnow2_{stat}','-', 'WorldClim'))
    
    return l_values, l_index

#### Fix HydroLAKES

In [28]:
l_values_lakes = [[] for _ in range(len(l_comids_geo))]
l_index_lakes = []

In [29]:
dataset = 'hydrolakes'
l_values_lakes, l_index_lakes, l_comids_lakes = attributes_from_hydrolakes(geo_folder, dataset, shp, ea_crs, l_values_lakes, l_index_lakes)

In [30]:
# create the hydrolakes attribute dataframe
att_df_lakes = pd.DataFrame(data = l_values_lakes, index = l_comids_lakes).transpose()
multi_index = pd.MultiIndex.from_tuples(l_index_lakes, names=['Category', 'Attribute', 'Unit', 'Source'])
att_df_lakes.index = multi_index

In [16]:
def subset_hydrolakes_to_subbasin(lakes, subbasin, ea_crs):

    '''lakes: GeoDataframe | subbasin: geometry | ea_crs: string'''

    # Clip the lakes polygon to the subbasin
    poly_lakes = gpd.clip(lakes,subbasin)

    # If we are left with any lake polygons:
    if len(poly_lakes) > 0 :
    # Update the Lake_area [km2] and Vol_total [million~m^3] values
    # This is needed in cases where due to clipping the lake polygon we end up with a partial lake in this subbasin
    
        # Old values
        old_areas = poly_lakes['Lake_area'] # km2
        old_volumes = poly_lakes['Vol_total'] # million m3
        
        # Get new area
        new_areas = poly_lakes.to_crs(ea_crs).area / 10**6 # [m2] / 10^6 = [km2]
        new_areas_rounded = round(new_areas,2) # this matches the number of significant digits in the test case (CAN_01DJ005)
        
        # Scale volume by new area - this is a bit simplistic but we have no better way to estimate the volume
        new_volumes = old_volumes * (new_areas_rounded / old_areas)
        
        # Replace values
        poly_lakes['Lake_area'] = new_areas_rounded
        poly_lakes['Vol_total'] = new_volumes

    return poly_lakes

In [17]:
def attributes_from_hydrolakes(geo_folder, dataset, basin_shp_path, ea_crs, l_values, l_index):
    
    '''Calculates open water attributes from HydroLAKES'''

    # We need some special actions if we're dealing with the distributed case, so check that first
    case = 'lumped'
    if 'distributed' in basin_shp_path:
        case = 'distributed'
    
    # Define the standard file name
    lake_str = str(geo_folder / dataset / 'raw' / 'HydroLAKES_polys_v10_NorthAmerica.shp')

    # Now handle the different cases
    if case == 'lumped':

        # Check if the file exists (some basins won't have lakes)
        if os.path.exists(lake_str):
            lakes = gpd.read_file(lake_str)
            res_mask  = lakes['Lake_type'] == 2 # Lake Type 2 == reservoir; see docs (https://data.hydrosheds.org/file/technical-documentation/HydroLAKES_TechDoc_v10.pdf)
            num_lakes = len(lakes)
            num_resvr = res_mask.sum()

        else: # no lakes shapefile
            lakes = None # this tells get_open_water_stats() that we had no lake shapefile
            num_lakes = 0
            num_resvr = 0

        l_values.append(num_lakes)
        l_index.append(('Open water', 'open_water_number',  '-', 'HydroLAKES'))
        l_values.append(num_resvr)
        l_index.append(('Open water', 'known_reservoirs',  '-', 'HydroLAKES'))

        # Summary stats
        l_values, l_index = get_open_water_stats(lakes, 'Lake_area', 'all', l_values, l_index) # All open water
        l_values, l_index = get_open_water_stats(lakes, 'Vol_total', 'all', l_values, l_index)
        l_values, l_index = get_open_water_stats(lakes, 'Lake_area', 'reservoir', l_values, l_index) # Reservoirs only
        l_values, l_index = get_open_water_stats(lakes, 'Vol_total', 'reservoir', l_values, l_index)

        # set the remaing output
        l_comids_lakes = None

    elif case == 'distributed':

        # Load the basin shape 
        basin = gpd.read_file(basin_shp_path)
        
        # Load the lake shape if we have one
        if os.path.exists(lake_str):
            lakes = gpd.read_file(lake_str)
        else:
            lakes = None # we use this to indicate we had no lakes at all
            
        # Loop over the individual polygons and create a new mini-HydroLAKES geodataframe for each polygon
        num_poly = len(basin)
        l_comids_lakes = basin['COMID'].values
        for i_poly in range(num_poly):

            # Rest the storage lists
            tmp_values = []
            tmp_index = []

            # subset the 'lakes' gdf to each individual polygon if we have 'lakes'           
            if lakes is not None:
                poly_lakes = subset_hydrolakes_to_subbasin(lakes, basin.iloc[i_poly]['geometry'], ea_crs)

                # Now see if we have a lake at all, and act accordingly
                if len(poly_lakes) > 0:
                    res_mask  = lakes['Lake_type'] == 2 # Lake Type 2 == reservoir; see docs (https://data.hydrosheds.org/file/technical-documentation/HydroLAKES_TechDoc_v10.pdf)
                    num_lakes = len(lakes)
                    num_resvr = res_mask.sum()
        
                else: # no lakes in this subbasins
                    poly_lakes = None # this tells get_open_water_stats() that we had no lake shapefile
                    num_lakes = 0
                    num_resvr = 0
            elif lakes is None: # this means we didn't have a lakes shapefile for this basin at all
                poly_lakes = None # this tells get_open_water_stats() we had nothing
                num_lakes = 0
                num_resvr = 0

            # Stats
            tmp_values.append(num_lakes)
            tmp_index.append(('Open water', 'open_water_number',  '-', 'HydroLAKES'))
            tmp_values.append(num_resvr)
            tmp_index.append(('Open water', 'known_reservoirs',  '-', 'HydroLAKES'))
            tmp_values, tmp_index = get_open_water_stats(poly_lakes, 'Lake_area', 'all', tmp_values, tmp_index) # All open water
            tmp_values, tmp_index = get_open_water_stats(poly_lakes, 'Vol_total', 'all', tmp_values, tmp_index)
            tmp_values, tmp_index = get_open_water_stats(poly_lakes, 'Lake_area', 'reservoir', tmp_values, tmp_index) # Reservoirs only
            tmp_values, tmp_index = get_open_water_stats(poly_lakes, 'Vol_total', 'reservoir', tmp_values, tmp_index)

            # Add the new values for this subbasin into the main l_values list
            l_values[i_poly] = tmp_values # This works because we create a unique dataframe just for HydroLAKES results

        # End of subbasin loop, add the new index entries only once
        l_index = tmp_index # we don't use append() in the distributed case because we'll be storing this in a dedicated HydroLAKES attribute df

    return l_values, l_index, l_comids_lakes

#### Fix the GLHYMPS case

In [31]:
l_values_glhymps = [[] for _ in range(len(l_comids_geo))]
l_index_glhymps = []

In [32]:
dataset = 'glhymps'

In [33]:
l_values_glhymps, l_index_glhymps, l_comids_glhymps = attributes_from_glhymps(geo_folder, dataset, shp, l_values_glhymps, l_index_glhymps, equal_area_crs=ea_crs)

/opt/software/custom/software/2020/avx2/MPI/gcc9/openmpi4/geo-stack/2022a/lib/python3.10/site-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,


In [34]:
# create the GLHYMPS attribute dataframe
att_df_glhymps = pd.DataFrame(data = l_values_glhymps, index = l_comids_glhymps).transpose()
multi_index = pd.MultiIndex.from_tuples(l_index_glhymps, names=['Category', 'Attribute', 'Unit', 'Source'])
att_df_glhymps.index = multi_index

In [18]:
def subset_glhymps_to_subbasin(glhymps, subbasin, ea_crs):

    poly_glhymps = gpd.clip(glhymps,subbasin)
    poly_glhymps['New_area_m2'] = poly_glhymps.to_crs(ea_crs).area

    return poly_glhymps

In [19]:
def attributes_from_glhymps(geo_folder, dataset, basin_shp_path, l_values, l_index, equal_area_crs='ESRI:102008'):

    '''Calculates attributes from GLHYMPS'''

    # We need some special actions if we're dealing with the distributed case, so check that first
    case = 'lumped'
    if 'distributed' in basin_shp_path:
        case = 'distributed'
    
    # Load the geology file
    geol_str = str(geo_folder / dataset / 'raw' / 'glhymps.shp')
    geol = gpd.read_file(geol_str)

    # -- File updates
    # Rename the columns, because the shortened ones are not very helpful
    if ('Porosity' in geol.columns) and ('Permeabili' in geol.columns) \
    and ('Permeabi_1' in geol.columns) and ('Permeabi_2' in geol.columns):
        geol.rename(columns={'Porosity': 'porosity',   'Permeabili': 'logK_Ferr',
                             'Permeabi_1': 'logK_Ice', 'Permeabi_2': 'logK_std'}, inplace=True)
        geol.to_file(geol_str)

    # Ensure we have the correct areas to work with
    if 'New_area_m2' not in geol.columns:
        geol['New_area_m2'] = geol.to_crs(equal_area_crs).area
        #geol.to_file(geol_str)

    # Clean up a few unsightly processing errors
    geol['Shape_Area'] = geol.to_crs(equal_area_crs).area
    geol = geol[['IDENTITY_','Shape_Area','porosity','logK_Ferr','logK_Ice','logK_std','geometry']]
    geol.to_file(geol_str)
    # -- End file updates

    # Now handle the different cases
    if case == 'lumped':
    
        # Create areal averages and standard deviations
        # Stdev source: https://stats.stackexchange.com/a/6536
        porosity_mean = (geol['porosity']*geol['New_area_m2']).sum()/geol['New_area_m2'].sum()
        num_obs_coef = ((geol['New_area_m2'] > 0).sum()-1)/(geol['New_area_m2'] > 0).sum()
        porosity_std = np.sqrt((geol['New_area_m2'] * (geol['porosity']-porosity_mean)**2).sum() / geol['New_area_m2'].sum())
        
        # Porosity
        l_values.append( geol['porosity'].min() )
        l_index.append(('Geology', 'porosity_min',  '-', 'GLHYMPS'))
        l_values.append( porosity_mean ) # areal average
        l_index.append(('Geology', 'porosity_mean',  '-', 'GLHYMPS'))
        l_values.append( geol['porosity'].max() )
        l_index.append(('Geology', 'porosity_max',  '-', 'GLHYMPS'))
        l_values.append( porosity_std )
        l_index.append(('Geology', 'porosity_std',  '-', 'GLHYMPS'))
    
        # Create areal averages and standard deviations
        # Stdev source: https://stats.stackexchange.com/a/6536
        permeability_mean = (geol['logK_Ice']*geol['New_area_m2']).sum()/geol['New_area_m2'].sum()
        num_obs_coef = ((geol['New_area_m2'] > 0).sum()-1)/(geol['New_area_m2'] > 0).sum()
        permeability_std = np.sqrt((geol['New_area_m2'] * (geol['logK_Ice']-permeability_mean)**2).sum() / geol['New_area_m2'].sum())
        
        # Permeability
        l_values.append( geol['logK_Ice'].min() )
        l_index.append(('Geology', 'log_permeability_min',  'm^2', 'GLHYMPS'))
        l_values.append( permeability_mean )
        l_index.append(('Geology', 'log_permeability_mean',  'm^2', 'GLHYMPS'))
        l_values.append( geol['logK_Ice'].max() )
        l_index.append(('Geology', 'log_permeability_max',  'm^2', 'GLHYMPS'))
        l_values.append( permeability_std )
        l_index.append(('Geology', 'log_permeability_std',  'm^2', 'GLHYMPS'))

        # Set the remaining output
        l_comids_glhymps = None

    # Case 2
    elif case == 'distributed':
    
        # Load the basin shape 
        basin = gpd.read_file(basin_shp_path)

        # Loop over the individual polygons and create a new mini-HydroLAKES geodataframe for each polygon
        num_poly = len(basin)
        l_comids_glhymps = basin['COMID'].values
        for i_poly in range(num_poly):

            # Rest the storage lists
            tmp_values = []
            tmp_index = []

            # Subset the shape to the subbasin
            poly_glhymps = subset_glhymps_to_subbasin(geol, basin.iloc[i_poly]['geometry'], equal_area_crs)

            # Create areal averages and standard deviations
            # Stdev source: https://stats.stackexchange.com/a/6536
            porosity_mean = (poly_glhymps['porosity']*poly_glhymps['New_area_m2']).sum()/poly_glhymps['New_area_m2'].sum()
            num_obs_coef = ((poly_glhymps['New_area_m2'] > 0).sum()-1)/(poly_glhymps['New_area_m2'] > 0).sum()
            porosity_std = np.sqrt((poly_glhymps['New_area_m2'] * (poly_glhymps['porosity']-porosity_mean)**2).sum() / poly_glhymps['New_area_m2'].sum())
            
            # Porosity
            tmp_values.append( poly_glhymps['porosity'].min() )
            tmp_index.append(('Geology', 'porosity_min',  '-', 'GLHYMPS'))
            tmp_values.append( porosity_mean ) # areal average
            tmp_index.append(('Geology', 'porosity_mean',  '-', 'GLHYMPS'))
            tmp_values.append( poly_glhymps['porosity'].max() )
            tmp_index.append(('Geology', 'porosity_max',  '-', 'GLHYMPS'))
            tmp_values.append( porosity_std )
            tmp_index.append(('Geology', 'porosity_std',  '-', 'GLHYMPS'))
        
            # Create areal averages and standard deviations
            # Stdev source: https://stats.stackexchange.com/a/6536
            permeability_mean = (poly_glhymps['logK_Ice']*poly_glhymps['New_area_m2']).sum()/poly_glhymps['New_area_m2'].sum()
            num_obs_coef = ((poly_glhymps['New_area_m2'] > 0).sum()-1)/(poly_glhymps['New_area_m2'] > 0).sum()
            permeability_std = np.sqrt((poly_glhymps['New_area_m2'] * (poly_glhymps['logK_Ice']-permeability_mean)**2).sum() / poly_glhymps['New_area_m2'].sum())
            
            # Permeability
            tmp_values.append( poly_glhymps['logK_Ice'].min() )
            tmp_index.append(('Geology', 'log_permeability_min',  'm^2', 'GLHYMPS'))
            tmp_values.append( permeability_mean )
            tmp_index.append(('Geology', 'log_permeability_mean',  'm^2', 'GLHYMPS'))
            tmp_values.append( poly_glhymps['logK_Ice'].max() )
            tmp_index.append(('Geology', 'log_permeability_max',  'm^2', 'GLHYMPS'))
            tmp_values.append( permeability_std )
            tmp_index.append(('Geology', 'log_permeability_std',  'm^2', 'GLHYMPS'))
            

            # Add the new values for this subbasin into the main l_values list
            l_values[i_poly] = tmp_values # This works because we create a unique dataframe just for HydroLAKES results

        # End of subbasin loop, add the new index entries only once
        l_index = tmp_index # we don't use append() in the distributed case because we'll be storing this in a dedicated HydroLAKES attribute df
    
    return l_values, l_index, l_comids_glhymps

#### Fix the RDRS distributed case

In [35]:
shp_path = shp

In [36]:
# trial the function
l_values_met = []
l_index_met = []
l_values_met, l_index_met, l_comids_met, _ = attributes_from_rdrs(met_folder, shp_path, 'RDRS', l_values_met, l_index_met)

Running distributed case for RDRS data.
Running subbasin 0
Running subbasin 1
Running subbasin 2


In [37]:
# make the forcing attributes dataframe
att_df_met = pd.DataFrame(data = l_values_met, index = l_comids_met).transpose()
multi_index = pd.MultiIndex.from_tuples(l_index_met, names=['Category', 'Attribute', 'Unit', 'Source'])
att_df_met.index = multi_index

In [20]:
def calculate_temp_prcp_stats(var, condition, hilo, l_values,l_index,
                              dataset='ERA5', units='hours'):
    
    '''Calculates frequency (mean) and duration (mean, median, skew, kurtosis) 
        of temperature/precipitation periods'''

    # Constants. We want everything in [days] for consistency with original CAMELS
    hours_per_day = 24 # [hours day-1]
    days_per_year = 365.25 # [days year-1]

    # Calculate frequencies
    freq = condition.mean(dim='time') * days_per_year # [-] * [days year-1]
    l_values.append(freq.values)
    l_index.append( ('Climate', f'{hilo}_{var}_freq', 'days year^-1', dataset) )
    
    # Calculate duration statistics
    durations = find_durations(condition) # [time steps]
    if units == 'hours':
        durations = durations / hours_per_day # [days] = [hours] / [hours day-1]
    l_values.append(np.mean(durations)) # [days]
    l_index.append( ('Climate', f'{hilo}_{var}_dur_mean', 'days', dataset) ) # Consistency with
    l_values.append(np.median(durations)) # [days]
    l_index.append( ('Climate', f'{hilo}_{var}_dur_median', 'days', dataset) )
    l_values.append(skew(durations)) # [-]
    l_index.append( ('Climate', f'{hilo}_{var}_dur_skew', '-', dataset) )
    l_values.append(kurtosis(durations)) # [-]
    l_index.append( ('Climate', f'{hilo}_{var}_dur_kurtosis', '-', dataset) )

    # Calculate timing statistic
    condition['season'] = ('time', [get_season(month) for month in condition['time.month'].values]) # add seasons
    season_groups = condition.groupby('season')
    season_list   = list(season_groups.groups.keys())
    max_season_id = int(season_groups.sum().argmax(dim='season').values) # find season with most True values
    l_values.append(season_list[max_season_id]) # add season abbrev
    l_index.append( ('Climate', f'{hilo}_{var}_timing', 'season', dataset) )

    return l_values, l_index

In [21]:
def attributes_from_rdrs(met_folder, shp_path, dataset, l_values, l_index, use_mfdataset=False):

    '''Calculates a variety of metrics from RDRS data'''

    # Define file locations, depending on if we are dealing with lumped or distributed cases
    if 'lumped' in shp_path:
        rdrs_folder = met_folder / 'lumped'
        case = 'lumped'
    elif 'distributed' in shp_path:
        rdrs_folder = met_folder / 'distributed'
        case = 'distributed'
    rdrs_files = sorted( glob.glob( str(rdrs_folder / 'RDRS_*.nc') ) )
    print(f'Running {case} case for RDRS data.')

    # Open the data
    if use_mfdataset or (case == 'distributed'):
        ds = xr.open_mfdataset(rdrs_files, combine="by_coords") # Don't use 'decode_cf=False' > this somehow loses most of the timesteps
    else:
        ds = xr.merge([xr.open_dataset(f) for f in rdrs_files])
        ds = ds.load()
        ds = ds.isel(hru=0) # We need this so the lumped and distributed cases have the same dimensions inside the calculation function: (time,nbnds)
                            # Without this, the lumped case has an extra 'hru' dimension (length 1) and this complicates value extraction

    # --- Act according to case
    if case == 'lumped':
        # -- Get the precipitation for hydrologic signature calculations later; this avoids having to reload the entire dataset another time
        ds_precip = ds['RDRS_v2.1_A_PR0_SFC'].copy()

        # --- Calculate the statistics
        l_values, l_index = calculate_rdrs_stats_from_ds(ds,l_values,l_index)
        return l_values, l_index, ds_precip, ds
    
    elif case == 'distributed':

        comid_order = [] # we need these later, to ensure we line the forcing attributes up with the geospatial attributes correctly

        # Loop over the HRUs
        num_hru = len(ds['hru'])
        for i in range(0,num_hru):

            print(f'Running subbasin {i}')

            # Load the data for this sub-basin
            ds_hru = ds.isel(hru=i) # example
            ds_hru.load() # Load data into memory; done in place

            # Specifically track the hruID (COMID) so we can ensure correct matches with the rest of the attributes later
            assert (ds_hru['hruId'].values == ds_hru['hruId'][0].values).all(), f"COMIDs not all identical {ds_hru['hruId'].values}"
            comid_order.append(ds_hru['hruId'][0].values)

            # calculate the stats
            tmp_values = [] # statistics for this subbasin - we need a single empty list so we can properly store in the nested list later
            tmp_index  = [] # we also store the index in a tmp variable, so we only get this once instead of appending num_hru times to the main index list
            tmp_values, tmp_index = calculate_rdrs_stats_from_ds(ds_hru,tmp_values,tmp_index) 
            l_values.append(tmp_values)

        # prep outputs
        comid_order = [arr.tolist() for arr in comid_order] # convert list of arrays to simple list
        l_index.append(tmp_index) # need this only once; tmp_index is already a list so this creates a nested list: [['Climate', 'num_years_rdrs', 'years', 'RDRS')]]
        return l_values, l_index[0], comid_order, ds # we need l_index[0] to return a non-nested list

In [22]:
def calculate_rdrs_stats_from_ds(ds,l_values,l_index):

    # Define various conversion constants
    water_density = 1000 # kg m-3
    mm_per_m = 1000 # mm m-1
    seconds_per_hour = 60*60 # s h-1
    seconds_per_day = seconds_per_hour*24 # s d-1
    days_per_month = np.array([31, 28.25, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]).reshape(-1, 1).flatten() # d month-1 || NOTE: added the .flatten() for distributed case, not tested for lumped
    days_per_year = days_per_month.sum()
    flip_sign = -1 # -; used to convert PET from negative (by convention this indicates an upward flux) to positive (ERA5 only)
    kelvin_to_celsius = -273.15
    pa_per_kpa = 1000 # Pa kPa-1
    
    # Select whole years only
    #   This avoids issues in cases where we have incomplete whole data years
    #   (e.g. 2000-06-01 to 2007-12-31) in basins with very seasonal weather
    #   (e.g. all precip occurs in Jan, Feb, Mar). By using only full years
    #   we avoid accidentally biasing the attributes.
    ds = subset_dataset_to_max_full_years(ds)
    num_years = len(ds.groupby('time.year'))
    l_values.append(num_years)
    l_index.append( ('Climate', 'num_years_rdrs', 'years', 'RDRS') )

    # --- Annual statistics (P, PET, T, aridity, seasonality, temperature, snow)
    # P
    yearly_pr0 = ds['RDRS_v2.1_A_PR0_SFC'].resample(time='1Y').mean() * seconds_per_day * days_per_year * mm_per_m / water_density # kg m-2 s-1 > mm yr-1
    l_values.append(yearly_pr0.mean().values)
    l_index.append(('Climate', 'PR0_SFC_mean', 'mm', 'RDRS'))
    l_values.append(yearly_pr0.std().values)
    l_index.append(('Climate', 'PR0_SFC_std', 'mm', 'RDRS'))

    # PET
    yearly_pet = ds['pet'].resample(time='1Y').mean() * seconds_per_day * days_per_year * mm_per_m / water_density # kg m-2 s-1 > mm yr-1
    l_values.append(yearly_pet.mean().values)
    l_index.append(('Climate', 'pet_mean', 'mm', 'RDRS'))
    l_values.append(yearly_pet.std().values)
    l_index.append(('Climate', 'pet_std', 'mm', 'RDRS'))

    # T
    yearly_tt = ds['RDRS_v2.1_P_TT_1.5m'].resample(time='1Y').mean() + kelvin_to_celsius # K > C
    l_values.append(yearly_tt.mean().values)
    l_index.append(('Climate', 'TT_mean', 'C', 'RDRS'))
    l_values.append(yearly_tt.std().values)
    l_index.append(('Climate', 'TT_std', 'C', 'RDRS'))

    # Aridity
    yearly_ari  = yearly_pet / yearly_pr0
    l_values.append(yearly_ari.mean().values)
    l_index.append(('Climate', 'aridity1_mean', '-', 'RDRS'))
    l_values.append(yearly_ari.std().values)
    l_index.append(('Climate', 'aridity1_std', '-', 'RDRS'))

    # Snow
    ds['snow'] = xr.where(ds['RDRS_v2.1_P_TT_1.5m'] < 273.15, ds['RDRS_v2.1_A_PR0_SFC'],0)
    yearly_snow = ds['snow'].resample(time='1Y').mean() * seconds_per_day * days_per_year * mm_per_m / water_density
    yearly_fs = yearly_snow / yearly_pr0
    l_values.append(yearly_fs.mean().values)
    l_index.append(('Climate', 'fracsnow1_mean', '-', 'RDRS'))
    l_values.append(yearly_fs.std().values)
    l_index.append(('Climate', 'fracsnow1_std', '-', 'RDRS'))

    # Seasonality
    seasonality = find_climate_seasonality_rdrs(ds,use_typical_cycle=False)
    l_values.append(seasonality.mean())
    l_index.append(('Climate', 'seasonality1_mean', '-', 'RDRS'))
    l_values.append(seasonality.std())
    l_index.append(('Climate', 'seasonality1_std', '-', 'RDRS'))

    # --- Monthly attributes
    # Calculate monthly PET in mm
    #      kg m-2 s-1 / kg m-3
    # mm month-1 = kg m-2 s-1 * kg-1 m3 * s d-1 * d month-1 * mm m-1 * -
    monthly_pet = ds['pet'].resample(time='1M').mean().groupby('time.month')
    pet_m = monthly_pet.mean() / water_density * seconds_per_day * days_per_month * mm_per_m  # [kg m-2 s-1] to [mm month-1]; negative to indicate upward flux
    pet_s = monthly_pet.std() / water_density * seconds_per_day * days_per_month * mm_per_m  # [kg m-2 s-1] to [mm month-1]; negative to indicate upward flux
    l_values, l_index = process_monthly_means_to_lists(pet_m, 'mean', l_values, l_index, 'pet', 'mm', source='RDRS')
    l_values, l_index = process_monthly_means_to_lists(pet_s, 'std', l_values, l_index, 'pet', 'mm', source='RDRS')

    # Same for precipitation: [mm month-1]
    monthly_pr0 = ds['RDRS_v2.1_A_PR0_SFC'].resample(time='1M').mean().groupby('time.month')
    pr0_m = monthly_pr0.mean() / water_density * seconds_per_day * days_per_month * mm_per_m # [kg m-2 s-1] to [mm month-1]
    pr0_s = monthly_pr0.std() / water_density * seconds_per_day * days_per_month * mm_per_m # [kg m-2 s-1] to [mm month-1]
    l_values, l_index = process_monthly_means_to_lists(pr0_m, 'mean', l_values, l_index, 'RDRS_v2.1_A_PR0_SFC', 'mm', source='RDRS')
    l_values, l_index = process_monthly_means_to_lists(pr0_s, 'std', l_values, l_index, 'RDRS_v2.1_A_PR0_SFC', 'mm', source='RDRS')

    # Monthly temperature statistics [C]
    monthly_tavg = (ds['RDRS_v2.1_P_TT_1.5m'].resample(time='1D').mean().resample(time='1M').mean() + kelvin_to_celsius).groupby('time.month')
    tavg_m = monthly_tavg.mean()
    tavg_s = monthly_tavg.std()
    l_values, l_index = process_monthly_means_to_lists(tavg_m, 'mean', l_values, l_index, 'tdavg', 'C', source = 'RDRS')
    l_values, l_index = process_monthly_means_to_lists(tavg_s, 'std', l_values, l_index, 'tdavg', 'C', source = 'RDRS')

    monthly_tmin = (ds['RDRS_v2.1_P_TT_1.5m'].resample(time='1D').min().resample(time='1M').mean() + kelvin_to_celsius).groupby('time.month')
    tmin_m = monthly_tmin.mean()
    tmin_s = monthly_tmin.std()
    l_values, l_index = process_monthly_means_to_lists(tmin_m, 'mean', l_values, l_index, 'tdmin', 'C', source = 'RDRS')
    l_values, l_index = process_monthly_means_to_lists(tmin_m, 'std', l_values, l_index, 'tdmin', 'C', source = 'RDRS')
    
    monthly_tmax = (ds['RDRS_v2.1_P_TT_1.5m'].resample(time='1D').max().resample(time='1M').mean() + kelvin_to_celsius).groupby('time.month')
    tmax_m = monthly_tmax.mean()
    tmax_s = monthly_tmax.std()
    l_values, l_index = process_monthly_means_to_lists(tmax_m, 'mean', l_values, l_index, 'tdmax', 'C', source = 'RDRS')
    l_values, l_index = process_monthly_means_to_lists(tmax_s, 'std', l_values, l_index, 'tdmax', 'C', source = 'RDRS')

    # Monthly shortwave and longwave [W m-2]
    monthly_sw = ds['RDRS_v2.1_P_FB_SFC'].resample(time='1M').mean().groupby('time.month')
    sw_m = monthly_sw.mean()
    sw_s = monthly_sw.std()
    l_values, l_index = process_monthly_means_to_lists(sw_m, 'mean', l_values, l_index, 'RDRS_v2.1_P_FB_SFC', 'W m^-2', source = 'RDRS')
    l_values, l_index = process_monthly_means_to_lists(sw_s, 'std', l_values, l_index, 'RDRS_v2.1_P_FB_SFC', 'W m^-2', source = 'RDRS')
    
    monthly_lw = ds['RDRS_v2.1_P_FI_SFC'].resample(time='1M').mean().groupby('time.month')
    lw_m = monthly_lw.mean(dim='time')
    lw_s = monthly_lw.std(dim='time')
    l_values, l_index = process_monthly_means_to_lists(lw_m, 'mean', l_values, l_index, 'RDRS_v2.1_P_FI_SFC', 'W m^-2', source = 'RDRS')
    l_values, l_index = process_monthly_means_to_lists(lw_s, 'std', l_values, l_index, 'RDRS_v2.1_P_FI_SFC', 'W m^-2', source = 'RDRS')

    # Surface pressure [Pa]
    monthly_sp = ds['RDRS_v2.1_P_P0_SFC'].resample(time='1M').mean().groupby('time.month')
    sp_m = monthly_sp.mean() / pa_per_kpa # [Pa] > [kPa]
    sp_s = monthly_sp.std() / pa_per_kpa
    l_values, l_index = process_monthly_means_to_lists(sp_m, 'mean', l_values, l_index, 'RDRS_v2.1_P_P0_SFC', 'kPa', source = 'RDRS')
    l_values, l_index = process_monthly_means_to_lists(sp_s, 'std', l_values, l_index, 'RDRS_v2.1_P_P0_SFC', 'kPa', source = 'RDRS')

    # Humidity [-]
    monthly_q = ds['RDRS_v2.1_P_HU_1.5m'].resample(time='1M').mean().groupby('time.month') # specific
    q_m = monthly_q.mean()
    q_s = monthly_q.std()
    l_values, l_index = process_monthly_means_to_lists(q_m, 'mean', l_values, l_index, 'RDRS_v2.1_P_HU_1.5m', 'kg kg^-1', source = 'RDRS')
    l_values, l_index = process_monthly_means_to_lists(q_s, 'std', l_values, l_index, 'RDRS_v2.1_P_HU_1.5m', 'kg kg^-1', source = 'RDRS')
    
    monthly_rh = ds['RDRS_v2.1_P_HR_1.5m'].resample(time='1M').mean().groupby('time.month') # relative
    rh_m = monthly_rh.mean()
    rh_s = monthly_rh.std()
    l_values, l_index = process_monthly_means_to_lists(rh_m, 'mean', l_values, l_index, 'RDRS_v2.1_P_HR_1.5m', 'kPa kPa^-1', source = 'RDRS')
    l_values, l_index = process_monthly_means_to_lists(rh_s, 'std', l_values, l_index, 'RDRS_v2.1_P_HR_1.5m', 'kPa kPa^-1', source = 'RDRS')

    # Wind speed [m s-1]
    monthly_w = ds['RDRS_v2.1_P_UVC_10m'].resample(time='1M').mean().groupby('time.month')
    w_m = monthly_w.mean()
    w_s = monthly_w.std()
    l_values, l_index = process_monthly_means_to_lists(w_m, 'mean', l_values, l_index, 'RDRS_v2.1_P_UVC_10m', 'm s^-1', source = 'RDRS')
    l_values, l_index = process_monthly_means_to_lists(w_s, 'std', l_values, l_index, 'RDRS_v2.1_P_UVC_10m', 'm s^-1', source = 'RDRS')

    # Wind direction
    monthly_phi = ds['phi'].resample(time='1M').apply(circmean_group).groupby('time.month')
    phi_m = monthly_phi.apply(circmean_group)
    phi_s = monthly_phi.apply(circstd_group)
    l_values, l_index = process_monthly_means_to_lists(phi_m, 'mean', l_values, l_index, 'phi', 'degrees', source = 'RDRS')
    l_values, l_index = process_monthly_means_to_lists(phi_s, 'std', l_values, l_index, 'phi', 'degrees', source = 'RDRS')

    # aridity
    monthly_pet = ds['pet'].resample(time='1M').mean()
    monthly_pr0 = ds['RDRS_v2.1_A_PR0_SFC'].resample(time='1M').mean()
    if (monthly_pr0 == 0).any():
        print(f'--- WARNING: attributes_from_rdrs(): adding 1 mm to monthly precipitation to avoid divide by zero error in aridity calculation')
        monthly_pr0[(monthly_pr0 == 0).sel(hru=0)] = 1 / mm_per_m * water_density / (seconds_per_day * days_per_month.mean()) # [mm month-1] / [mm m-1] * [kg m-3] / ([s d-1] * [d month-1]) = [kg m-2 s-1]
    monthly_ari = (monthly_pet / monthly_pr0).groupby('time.month')
    ari_m = monthly_ari.mean()
    ari_s = monthly_ari.std()
    l_values, l_index = process_monthly_means_to_lists(ari_m, 'mean', l_values, l_index, 'aridity1', '-', source = 'RDRS')
    l_values, l_index = process_monthly_means_to_lists(ari_s, 'std', l_values, l_index, 'aridity1', '-', source = 'RDRS')

    # snow
    monthly_snow = ds['snow'].resample(time='1M').mean()
    monthly_pr0 = ds['RDRS_v2.1_A_PR0_SFC'].resample(time='1M').mean()
    if (monthly_pr0 == 0).any():
        print(f'--- WARNING: attributes_from_rdrs(): adding 1 mm to monthly precipitation to avoid divide by zero error in snow calculation. Note that by definition this cannot change the fraction snow result (if there is 0 precip, none of it will fall as snow)')
        monthly_pr0[(monthly_pr0 == 0).sel(hru=0)] = 1 / mm_per_m * water_density / (seconds_per_day * days_per_month.mean()) # [mm month-1] / [mm m-1] * [kg m-3] / ([s d-1] * [d month-1]) = [kg m-2 s-1]
    monthly_snow = (monthly_snow / monthly_pr0).groupby('time.month')
    fsnow_m = monthly_snow.mean()
    fsnow_s = monthly_snow.std()
    l_values, l_index = process_monthly_means_to_lists(fsnow_m, 'mean', l_values, l_index, 'fracsnow1', '-', source = 'RDRS')
    l_values, l_index = process_monthly_means_to_lists(fsnow_s, 'std', l_values, l_index, 'fracsnow1', '-', source = 'RDRS') 

    # --- High-frequency statistics (high/low duration/timing/magnitude)
    #  Everyone does precip. We'll add temperature too as a drought/frost indicator
    
    # -- LOW TEMPERATURE
    variable  = 'RDRS_v2.1_P_TT_1.5m'
    low_threshold = 273.15 # K, freezing point
    low_condition = ds[variable] < low_threshold
    l_values,l_index = calculate_temp_prcp_stats('temp',low_condition,'low',l_values,l_index, dataset='RDRS')

    # -- HIGH TEMPERATURE
    # WMO defines a heat wave as a 5-day or longer period with maximum daily temperatures 5C above 
    # "standard" daily max temperature (1961-1990; source:
    # https://www.ifrc.org/sites/default/files/2021-06/10-HEAT-WAVE-HR.pdf).
    # We define a "hot day" therefore as a day with a maximum temperature 5 degrees over the 
    # the long-term mean maximum temperature.
    #   Note: we don't have 1961-1990 data for some stations, so we stick with long-term mean.
    #   Note: this will in most cases slightly underestimate heat waves compared to WMO definition
    
    # First, we identify the long-term mean daily maximum temperature in a dedicated function
    var = 'RDRS_v2.1_P_TT_1.5m'
    high_threshold = create_mean_daily_max_series(ds,var=var)
    
    # Next, we check if which 't' values are 5 degrees above the long-term mean daily max 
    #  ("(ds['t'] > result_array + 5)"), and resample this to a daily time series 
    #  ("resample(time='1D')") filled with "True" if any value in that day was True.
    daily_flags = (ds[var] > high_threshold + 5).resample(time='1D').any()
    
    # Finally, we reindex these daily flags back onto the hourly time series by filling values
    high_condition = daily_flags.reindex_like(ds[var], method='ffill')
    
    # Now calculate stats like before
    l_values,l_index = calculate_temp_prcp_stats('temp',high_condition,'high',l_values,l_index, dataset='RDRS')

    # -- LOW PRECIPITATION
    variable = 'RDRS_v2.1_A_PR0_SFC'
    # We'll stick with the original CAMELS definition of low precipitation: < 1 mm day-1
    # It may not make too much sense to look at "dry hours" so we'll do this analysis at daily step
    low_threshold = 1 # [mm d-1]
    # Create daily precipitation sum (divided by density, times mm m-1 cancels out)
    # [kg m-2 s-1] * [s h-1] / [kg m-3] * [mm m-1] = [mm h-1]
    low_condition = (ds[variable] * seconds_per_hour).resample(time='1D').sum() < low_threshold
    l_values,l_index = calculate_temp_prcp_stats('prec',low_condition,'low',l_values,l_index,
                                             units='days', dataset='RDRS') # this 'units' argument prevents conversion to days inside the functiom
    
    # -- HIGH PRECIPITATION
    # CAMELS: > 5 times mean daily precip
    high_threshold = 5 * (ds[variable] * seconds_per_hour).resample(time='1D').sum().mean()
    high_condition = (ds[variable] * seconds_per_hour).resample(time='1D').sum() >= high_threshold
    l_values,l_index = calculate_temp_prcp_stats('prec',high_condition,'high',l_values,l_index,
                                                 units='days', dataset='RDRS')

    return l_values, l_index

#### Fix the MERIT distributed case

First, check if just the raster part completes OK

In [13]:
import os
import numpy as np
from scipy.stats import circmean, circstd
from rasterstats import zonal_stats

In [25]:
# -- lumped case for trial
l_values = []
l_index = []
basin = 'CAN_01DJ005'
shp_str = str(shp_lump_path)
riv_str = str(riv)

l_values_geo, l_index_geo,_ = attributes_from_merit(geo_folder, 'merit', 
                                                  shp_str, riv_str, row, 
                                                  l_values, l_index, equal_area_crs=ea_crs)

# Test with a fake second station
l_gauges = [basin,'dummy']

# Make the dataframe
input_dict = dict(zip(l_gauges, [l_values,l_values]))
df = pd.DataFrame(input_dict)

# Set the index
multi_index = pd.MultiIndex.from_tuples(l_index, names=['Category', 'Attribute', 'Unit', 'Source'])
df.index = multi_index

# Drop the fake extra column
df = df.drop(columns=['dummy'], axis=1)

df

NameError: name 'attributes_from_merit' is not defined

In [59]:
# Load the shapefile to get the sub-basin order for geotiffs and shapefiles
gdf = gpd.read_file(shp)
l_comids_geo = gdf['COMID'].to_list() # Store the polygon IDs into a list, we'll later use this as columns in the attribute dataframes

# Initialize storage lists for geotiff and shapefile attributes
l_values_geo = [[] for _ in range(len(l_comids_geo))] # Initialize an empty list where we'll store this basin's attributes - nested lists for each subbasin
l_index_geo = [] # Initialize an empty list where we'll store the attribute descriptions - this will be our dataframe index

In [65]:
dataset = 'merit'
shp_str = shp
riv_str = riv
l_values_geo, l_index_geo, l_comids_merit = attributes_from_merit(geo_folder, dataset, shp_str, riv_str, row, l_values_geo, l_index_geo, equal_area_crs=ea_crs)
assert (l_comids_geo == l_comids_merit).all(), f"mismatch between COMID orders determined before and inside attributes_from_merit()"

In [63]:
# create the geospatial attribute dataframe
att_df_geo = pd.DataFrame(data = l_values_geo, index = l_comids_geo).transpose()
multi_index = pd.MultiIndex.from_tuples(l_index_geo, names=['Category', 'Attribute', 'Unit', 'Source'])
att_df_geo.index = multi_index
att_df_geo

72048321  \
Category   Attribute        Unit     Source                                        
Topography centroid_lat     degrees  MERIT Hydro                       45.470122   
           centroid_lon     degrees  MERIT Hydro                      -63.619328   
           basin_area       km^2     MERIT Hydro                       35.054325   
           elev_min         m.a.s.l. MERIT Hydro                       25.400000   
           elev_mean        m.a.s.l. MERIT Hydro                      143.622503   
           elev_max         m.a.s.l. MERIT Hydro                      314.200012   
           elev_std         m.a.s.l. MERIT Hydro                       62.706802   
           slope_min        degrees  MERIT Hydro                        0.000000   
           slope_mean       degrees  MERIT Hydro                        4.318055   
           slope_max        degrees  MERIT Hydro                       24.456249   
           slope_std        degrees  MERIT Hydro                        2.860660   
           aspect_min       degrees  MERIT Hydro                        0.000000   
           aspect_mean      degrees  MERIT Hydro                      159.095487   
           aspect_max       degrees  MERIT Hydro                      359.999847   
           aspect_std       degrees  MERIT Hydro                       85.905128   
           centroid_lat     degrees  MERIT Hydro                       45.470122   
           centroid_lon     degrees  MERIT Hydro                      -63.619328   
           basin_area       km^2     MERIT Hydro                       35.054325   
           elev_min         m.a.s.l. MERIT Hydro                       25.400000   
           elev_mean        m.a.s.l. MERIT Hydro                      143.622503   
           elev_max         m.a.s.l. MERIT Hydro                      314.200012   
           elev_std         m.a.s.l. MERIT Hydro                       62.706802   
           slope_min        degrees  MERIT Hydro                        0.000000   
           slope_mean       degrees  MERIT Hydro                        4.318055   
           slope_max        degrees  MERIT Hydro                       24.456249   
           slope_std        degrees  MERIT Hydro                        2.860660   
           aspect_min       degrees  MERIT Hydro                        0.000000   
           aspect_mean      degrees  MERIT Hydro                      159.095487   
           aspect_max       degrees  MERIT Hydro                      359.999847   
           aspect_std       degrees  MERIT Hydro                       85.905128   
           stream_length    km       MERIT Hydro Basins                 5.192127   
           stream_slope     m m^-1   MERIT Hydro Basins                 0.004996   
           upstream_area    km^2     MERIT Hydro Basins               115.071409   
           stream_density   km^-1    MERIT Hydro, MERIT Hydro Basins    0.148117   
           elongation_ratio -        MERIT Hydro, MERIT Hydro Basins    1.286710   

                                                                        72049179  \
Category   Attribute        Unit     Source                                        
Topography centroid_lat     degrees  MERIT Hydro                       45.529858   
           centroid_lon     degrees  MERIT Hydro                      -63.600849   
           basin_area       km^2     MERIT Hydro                       29.332096   
           elev_min         m.a.s.l. MERIT Hydro                       59.900002   
           elev_mean        m.a.s.l. MERIT Hydro                      267.559147   
           elev_max         m.a.s.l. MERIT Hydro                      362.700012   
           elev_std         m.a.s.l. MERIT Hydro                       63.034259   
           slope_min        degrees  MERIT Hydro                        0.000000   
           slope_mean       degrees  MERIT Hydro                        4.923624   
           slope_max        degrees  ME

In [14]:
def get_river_attributes(riv_str, bas_str, l_values, l_index, area, equal_area_crs='ESRI:102008'):
    
    '''Calculates topographic attributes from a MERIT Hydro Basins river polygon'''

    # We need some special actions if we're dealing with the distributed case, so check that first
    case = 'lumped'
    if 'distributed' in bas_str:
        case = 'distributed'

    # Handle the cases
    if case == 'lumped': 
    
        # Initialize NaN for cases where we have no shapefile or an empty one
        stream_total = np.nan
        min_length = np.nan
        mean_length = np.nan
        max_length = np.nan
        std_length = np.nan
        riv_order = np.nan
        density = np.nan
        elongation = np.nan
        
        # Check if the file exists (for headwaters we won't have a river polygon)
        if os.path.exists(riv_str):
            
            # Load shapefiles
            river = gpd.read_file(riv_str)
            river = river.set_index('COMID')
            river = river[~river.index.duplicated(keep='first')] # Removes any duplicate river segments
    
            # Check if we actually have a river segment (empty shapefile is not the same as no shapefile)
            if len(river) > 0:
                # Raw data
                stream_lengths = []
                headwaters = river[(river['maxup'] == 0) | (river['maxup'].isna())] # identify reaches with no upstream
                for COMID in headwaters.index:
                    stream_length = 0
                    while COMID in river.index:
                        stream_length += river.loc[COMID]['new_len_km'] # Add the length of the current segment
                        COMID = river.loc[COMID]['NextDownID'] # Get the downstream reach
                    stream_lengths.append(stream_length) # If we get here we ran out of downstream IDs
            
                # Stats
                stream_total = river['new_len_km'].sum()
                stream_lengths = np.array(stream_lengths)
                min_length = stream_lengths.min()
                mean_length = stream_lengths.mean()
                max_length = stream_lengths.max()
                std_length = stream_lengths.std()
                riv_order = river['order'].max()
                density = stream_total/area
                elongation = 2*np.sqrt(area/np.pi)/max_length
    
        # Update lists
        l_values.append(min_length)
        l_index.append(('Topography', 'stream_length_min',  'km', 'MERIT Hydro Basins'))
        l_values.append(mean_length)
        l_index.append(('Topography', 'stream_length_mean', 'km', 'MERIT Hydro Basins'))
        l_values.append(max_length)
        l_index.append(('Topography', 'stream_length_max',  'km', 'MERIT Hydro Basins'))
        l_values.append(std_length)
        l_index.append(('Topography', 'stream_length_std',  'km', 'MERIT Hydro Basins'))
        l_values.append(stream_total)
        l_index.append(('Topography', 'segment_length_total', 'km', 'MERIT Hydro Basins'))
        
        # Order
        l_values.append(riv_order)
        l_index.append(('Topography', 'stream_order_max',  '-', 'MERIT Hydro Basins'))
    
        # Derived
        l_values.append(density)
        l_index.append(('Topography', 'stream_density',  'km^-1', 'MERIT Hydro, MERIT Hydro Basins'))
        l_values.append(elongation)
        l_index.append(('Topography', 'elongation_ratio','-', 'MERIT Hydro, MERIT Hydro Basins'))

        # Final empty output
        merit_comids = None

    # distributed case
    elif case == 'distributed':

        # Get the basin shape and figure out what we're dealing with
        basin = gpd.read_file(bas_str)
        num_poly = len(basin)
        merit_comids = basin['COMID'].values

        # Check if the file exists (for headwaters we won't have a river polygon)
        if os.path.exists(riv_str):
            
            # Load shapefiles
            river = gpd.read_file(riv_str)
            river = river.set_index('COMID')
            river = river[~river.index.duplicated(keep='first')] # Removes any duplicate river segments

            # Loop over the basins and process
            for i_poly in range(num_poly):

                # Find the river segment, if any, and get values for the statistics
                comid = basin.iloc[i_poly]['COMID']
                if comid in river.index:
                    riv_poly = river.loc[comid]
                    stream_length = riv_poly['new_len_km']
                    slope = riv_poly['slope']
                    upstream_area = riv_poly['uparea']
                    basin_area = basin.to_crs('ESRI:102008').area.iloc[0] / 10**6 # area in km2
                    density = stream_length/basin_area
                    elongation = 2*np.sqrt(basin_area/np.pi)/stream_length
                else:
                    stream_length = np.nan # no delineated river here, so no stats
                    slope = np.nan
                    upstream_area = np.nan
                    density = np.nan
                    elongation = np.nan
                
                # Append statistics to list
                l_values[i_poly].append(stream_length)
                l_values[i_poly].append(slope)
                l_values[i_poly].append(upstream_area)
                l_values[i_poly].append(density)
                l_values[i_poly].append(elongation)
                    
        else: # no river shapefile
            
            # just add nans everywhere
            stream_length = np.nan # no delineated river here, so no stats
            slope = np.nan
            upstream_area = np.nan
            density = np.nan
            elongation = np.nan
    
            for i_poly in range(num_poly):
                l_values[i_poly].append(stream_length)
                l_values[i_poly].append(slope)
                l_values[i_poly].append(upstream_area)
                l_values[i_poly].append(density)
                l_values[i_poly].append(elongation)

        # Update the index list
        l_index += [('Topography', 'stream_length',    'km',     'MERIT Hydro Basins'),
                    ('Topography', 'stream_slope',     'm m^-1', 'MERIT Hydro Basins'),
                    ('Topography', 'upstream_area',    'km^2',   'MERIT Hydro Basins'),
                    ('Topography', 'stream_density',   'km^-1',  'MERIT Hydro, MERIT Hydro Basins'),
                    ('Topography', 'elongation_ratio', '-',      'MERIT Hydro, MERIT Hydro Basins')]
    
    return l_values,l_index,merit_comids

In [15]:
def calc_circmean(x):
    return circmean(x.compressed(), high=360) # need compressed() because scipy < 1.12.0 circmean/std don't work well with masked arrays. This flattens the array and keeps values only

def calc_circstd(x):
    return circstd(x.compressed(), high=360)

In [16]:
def attributes_from_merit(geo_folder, dataset, shp_str, riv_str, row, l_values, l_index, equal_area_crs='ESRI:102008'):
    
    '''Calculates topographic attributes from MERIT data'''

    # We need some special actions if we're dealing with the distributed case, so check that first
    case = 'lumped'
    if 'distributed' in shp_str:
        case = 'distributed'

    # Load the shapefile
    basin = gpd.read_file(shp_str)

    ## Known values
    if case == 'lumped':
        clon = basin.to_crs(ea_crs).centroid.to_crs('EPSG:4326').x.iloc[0] # lon
        clat = basin.to_crs(ea_crs).centroid.to_crs('EPSG:4326').y.iloc[0] # lat       
        area = row['Basin_area_km2']
        slat = row['Station_lat']
        slon = row['Station_lon']
        src  = row['Station_source']
        l_values.append(clat)
        l_index.append(('Topography', 'centroid_lat',  'degrees', 'MERIT Hydro'))
        l_values.append(clon)
        l_index.append(('Topography', 'centroid_lon',  'degrees', 'MERIT Hydro'))
        l_values.append(area)
        l_index.append(('Topography', 'basin_area', 'km^2', 'MERIT Hydro'))
        l_values.append(slat)
        l_index.append(('Topography', 'gauge_lat',  'degrees', 'USGS/WSC'))
        l_values.append(slon)
        l_index.append(('Topography', 'gauge_lon',  'degrees', 'USGS/WSC'))
    
    elif case == 'distributed':
        num_poly = len(basin)
        for i_poly in range(num_poly):
            clon = basin.to_crs(ea_crs).centroid.to_crs('EPSG:4326').x.iloc[i_poly] # lon
            clat = basin.to_crs(ea_crs).centroid.to_crs('EPSG:4326').y.iloc[i_poly] # lat       
            area = (basin.to_crs(ea_crs).area / 10**6).iloc[i_poly]
            l_values[i_poly].append(clat)
            l_values[i_poly].append(clon)
            l_values[i_poly].append(area)    
        l_index.append(('Topography', 'centroid_lat',  'degrees', 'MERIT Hydro')) # these are outside the subbasin loop because we need them only once
        l_index.append(('Topography', 'centroid_lon',  'degrees', 'MERIT Hydro'))
        l_index.append(('Topography', 'basin_area', 'km^2', 'MERIT Hydro'))    

    ## RASTERS
    # Slope and elevation can use zonal stats built-ins
    files = [str(geo_folder / dataset / 'raw'    / 'merit_hydro_elv.tif'),
             str(geo_folder / dataset / 'slope'  / 'merit_hydro_slope.tif')]
    attrs = ['elev',     'slope']
    units = ['m.a.s.l.', 'degrees']
    stats = ['mean', 'min', 'max', 'std']
    for tif,att,unit in zip(files,attrs,units):
        zonal_out = zonal_stats(shp_str, tif, stats=stats)
        scale,offset = csa.read_scale_and_offset(tif)
        l_values = csa.update_values_list(l_values, stats, zonal_out, scale, offset)
        l_index += [('Topography', f'{att}_min',  f'{unit}', 'MERIT Hydro'),
                    ('Topography', f'{att}_mean', f'{unit}', 'MERIT Hydro'),
                    ('Topography', f'{att}_max',  f'{unit}', 'MERIT Hydro'),
                    ('Topography', f'{att}_std',  f'{unit}', 'MERIT Hydro')]

    # Aspect needs circular stats as custom input
    stats = ['min', 'max']
    tif = str(geo_folder / dataset / 'aspect' / 'merit_hydro_aspect.tif')
    zonal_out = zonal_stats(shp_str, tif, stats=stats, add_stats={'circ_mean': calc_circmean,
                                                                  'circ_std':  calc_circstd})
    scale,offset = csa.read_scale_and_offset(tif)
    l_values = csa.update_values_list(l_values, ['min', 'max', 'circ_mean', 'circ_std'], zonal_out, scale, offset)
    l_index += [('Topography', 'aspect_min',   'degrees', 'MERIT Hydro'),
                ('Topography', 'aspect_mean',  'degrees', 'MERIT Hydro'),
                ('Topography', 'aspect_max',   'degrees', 'MERIT Hydro'),
                ('Topography', 'aspect_std',   'degrees', 'MERIT Hydro')]
    #l_values, l_index = csa.get_aspect_attributes(tif,l_values,l_index)
    
    ## VECTOR
    l_values, l_index, merit_comids = get_river_attributes(riv_str, shp_str, l_values, l_index, area, equal_area_crs=equal_area_crs)
    
    return l_values, l_index, merit_comids

#### Merge the separate attribute dataframes

In [38]:
# check we have the same columns in all dfs
geo_columns = att_df_geo.columns.unique().sort_values()
met_columns = att_df_met.columns.unique().sort_values()
lak_columns = att_df_lakes.columns.unique().sort_values()
glh_columns = att_df_glhymps.columns.unique().sort_values()

assert (geo_columns == met_columns).all(), f"COMID mismatches between meteo and geospatial attribute dataframes"
assert (geo_columns == lak_columns).all(), f"COMID mismatches between hydrolakes and geospatial attribute dataframes"
assert (geo_columns == glh_columns).all(), f"COMID mismatches between glhymps and geospatial attribute dataframes"

In [39]:
att_df = pd.concat([att_df_met,att_df_geo,att_df_lakes,att_df_glhymps])

In [40]:
att_df

72048321.0  \
Category Attribute             Unit  Source                        
Climate  num_years_rdrs        years RDRS                     24   
         PR0_SFC_mean          mm    RDRS     1389.9713642913412   
         PR0_SFC_std           mm    RDRS     153.05992530337002   
         pet_mean              mm    RDRS     1026.7960842582397   
         pet_std               mm    RDRS      50.76377868009653   
...                                                          ...   
Geology  porosity_std          -     GLHYMPS            0.071502   
         log_permeability_min  m^2   GLHYMPS               -16.5   
         log_permeability_mean m^2   GLHYMPS           -14.54584   
         log_permeability_max  m^2   GLHYMPS               -12.5   
         log_permeability_std  m^2   GLHYMPS            1.067653   

                                                      72049179.0  \
Category Attribute             Unit  Source                        
Climate  num_years_rdrs        years RDRS                     24   
         PR0_SFC_mean          mm    RDRS     1383.0917654395307   
         PR0_SFC_std           mm    RDRS     155.11670134231477   
         pet_mean              mm    RDRS     1022.7861444815062   
         pet_std               mm    RDRS     48.809287944138774   
...                                                          ...   
Geology  porosity_std          -     GLHYMPS            0.049969   
         log_permeability_min  m^2   GLHYMPS               -15.2   
         log_permeability_mean m^2   GLHYMPS           -14.19256   
         log_permeability_max  m^2   GLHYMPS               -14.1   
         log_permeability_std  m^2   GLHYMPS            0.305366   

                                                      72049188.0  
Category Attribute             Unit  Source                       
Climate  num_years_rdrs        years RDRS                     24  
         PR0_SFC_mean          mm    RDRS     1393.9310812791518  
         PR0_SFC_std           mm    RDRS      153.5888911052156  
         pet_mean              mm    RDRS     1024.7584377462772  
         pet_std               mm    RDRS      50.45739506616813  
...                                                          ...  
Geology  porosity_std          -     GLHYMPS            0.056043  
         log_permeability_min  m^2   GLHYMPS               -16.5  
         log_permeability_mean m^2   GLHYMPS          -14.432358  
         log_permeability_max  m^2   GLHYMPS               -12.5  
         log_permeability_std  m^2   GLHYMPS            0.843112  

[747 rows x 3 columns]

In [ ]:
# Save to file

### Full code trials

In [8]:
# interesting ix's to try:
9  # 1 basin
13 # 10 basins (1 really small)
46 # 3 basins

46

In [8]:
print(debug_message)
for ix,row in cs_meta.iterrows():

    # DEBUGGING
    if ix != 13: continue

    # Get the paths
    basin_id, _, shp_dist_path, _, _ = prepare_delineation_outputs(cs_meta, ix, basins_path)
    geo_folder = basins_path / 'basin_data' / basin_id / 'geospatial'
    met_folder = basins_path / 'basin_data' / basin_id / 'forcing'
    hyd_folder = basins_path / 'basin_data' / basin_id / 'observations'

    # Make a temporary folder for storage
    temp_path = temp_path / basin_id 
    temp_path.mkdir(exist_ok=True, parents=True)

    # Define the shapefiles
    shp = str(shp_dist_path).format('basin') # because zonalstats wants a file path, not a geodataframe
    riv = str(shp_dist_path).format('river') # For topographic attributes

    # Specifically set the case
    case = 'distributed'

    # Load the shapefile to get the sub-basin order for geotiffs and shapefiles
    gdf = gpd.read_file(shp)

    # Data storage
    l_comids_geo = gdf['COMID'].to_list() # Store the polygon IDs into a list, we'll later use this as columns in the attribute dataframes
    l_values_geo = [[] for _ in range(len(l_comids_geo))] # Initialize an empty list where we'll store this basin's attributes - nested lists for each subbasin
    l_index_geo = [] # Initialize an empty list where we'll store the attribute descriptions - this will be our dataframe index

    l_values_lakes = [[] for _ in range(len(l_comids_geo))] # HydroLAKES open water bodies shapefile
    l_index_lakes = []
    l_values_glhymps = [[] for _ in range(len(l_comids_geo))] # GLHYMPS geology shapefile
    l_index_glhymps = []

    l_values_met = [] # RDRS meteorological netcdf data - creating a nested list with subbasins is done differently for RDRS, so this line is supposed to look different from the other l_values_[x] lists
    l_index_met = []

    # Data-specific processing
    print(f'Processing geospatial data into attributes for {basin_id}')
    for dataset in data_subfolders:
        print(f' - processing {dataset}')

        ## CLIMATE
        if dataset == 'rdrs':
            #l_values, l_index, ds_precip, ds_rdrs = csa.attributes_from_rdrs(met_folder, shp, dataset, l_values, l_index)
            l_values_met, l_index_met, l_comids_met, _ = csa.attributes_from_rdrs(met_folder, shp, dataset, l_values_met, l_index_met)
        if dataset == 'worldclim':
            csa.oudin_pet_from_worldclim(geo_folder, dataset) # Get an extra PET estimate to sanity check RDRS outcomes
            csa.aridity_and_fraction_snow_from_worldclim(geo_folder, dataset) # Get monthly aridity and fraction snow maps
            l_values_geo, l_index_geo = csa.attributes_from_worldclim(geo_folder, dataset, shp, l_values_geo, l_index_geo, case=case)

        ## LAND COVER
        if dataset == 'forest_height':
            l_values_geo, l_index_geo = csa.attributes_from_forest_height(geo_folder, dataset, shp, l_values_geo, l_index_geo, case=case)
        if dataset == 'lai':
            l_values_geo, l_index_geo = csa.attributes_from_lai(geo_folder, dataset, temp_path, shp, l_values_geo, l_index_geo, case=case)
        if dataset == 'glclu2019':
            l_values_geo, l_index_geo = csa.attributes_from_glclu2019(geo_folder, dataset, shp, l_values_geo, l_index_geo, case=case)
        if dataset == 'modis_land':
            l_values_geo, l_index_geo = csa.attributes_from_modis_land(geo_folder, dataset, shp, l_values_geo, l_index_geo, case=case)
        if dataset == 'lgrip30':
            l_values_geo, l_index_geo = csa.attributes_from_lgrip30(geo_folder, dataset, shp, l_values_geo, l_index_geo, case=case)

        ## TOPOGRAPHY
        if dataset == 'merit':
            l_values_geo, l_index_geo, l_comids_merit = csa.attributes_from_merit(geo_folder, dataset, shp, riv, row, l_values_geo, l_index_geo, equal_area_crs=ea_crs, case=case)
            assert (l_comids_geo == l_comids_merit).all(), f"mismatch between COMID orders determined before and inside attributes_from_merit()"

        ## OPENWATER
        if dataset == 'hydrolakes':
            #l_values, l_index = csa.attributes_from_hydrolakes(geo_folder, dataset, l_values, l_index)
            l_values_lakes, l_index_lakes, l_comids_lakes = csa.attributes_from_hydrolakes(geo_folder, dataset, shp, ea_crs, 
                                                                                           l_values_lakes, l_index_lakes)

        ## SOIL
        if dataset == 'pelletier':
            l_values_geo, l_index_geo = csa.attributes_from_pelletier(geo_folder, dataset, shp, l_values_geo, l_index_geo, case=case)
        if dataset == 'soilgrids':
            l_values_geo, l_index_geo = csa.attributes_from_soilgrids(geo_folder, dataset, shp, l_values_geo, l_index_geo, case=case)

        ## GEOLOGY
        if dataset == 'glhymps':
            #l_values, l_index = csa.attributes_from_glhymps(geo_folder, dataset, l_values, l_index)
            l_values_glhymps, l_index_glhymps, l_comids_glhymps = csa.attributes_from_glhymps(geo_folder, dataset, shp, 
                                                                                              l_values_glhymps, l_index_glhymps, 
                                                                                              equal_area_crs=ea_crs)

    ## MERGE THE DATAFRAMES
    # Make the individual dataframes
    # - RDRS
    att_df_met = pd.DataFrame(data = l_values_met, index = l_comids_met).transpose()
    multi_index = pd.MultiIndex.from_tuples(l_index_met, names=['Category', 'Attribute', 'Unit', 'Source'])
    att_df_met.index = multi_index

    # - geotiffs
    att_df_geo = pd.DataFrame(data = l_values_geo, index = l_comids_geo).transpose()
    multi_index = pd.MultiIndex.from_tuples(l_index_geo, names=['Category', 'Attribute', 'Unit', 'Source'])
    att_df_geo.index = multi_index
    
    # - HydroLAKES
    att_df_lakes = pd.DataFrame(data = l_values_lakes, index = l_comids_lakes).transpose()
    multi_index = pd.MultiIndex.from_tuples(l_index_lakes, names=['Category', 'Attribute', 'Unit', 'Source'])
    att_df_lakes.index = multi_index

    # - GLHYMPS
    att_df_glhymps = pd.DataFrame(data = l_values_glhymps, index = l_comids_glhymps).transpose()
    multi_index = pd.MultiIndex.from_tuples(l_index_glhymps, names=['Category', 'Attribute', 'Unit', 'Source'])
    att_df_glhymps.index = multi_index

    # Check we have the same columns in all dfs
    geo_columns = att_df_geo.columns.unique().sort_values()
    met_columns = att_df_met.columns.unique().sort_values()
    lak_columns = att_df_lakes.columns.unique().sort_values()
    glh_columns = att_df_glhymps.columns.unique().sort_values()
    assert (geo_columns == met_columns).all(), f"COMID mismatches between meteo and geospatial attribute dataframes"
    assert (geo_columns == lak_columns).all(), f"COMID mismatches between hydrolakes and geospatial attribute dataframes"
    assert (geo_columns == glh_columns).all(), f"COMID mismatches between glhymps and geospatial attribute dataframes"

    # Merge and save
    att_df = pd.concat([att_df_met,att_df_geo,att_df_lakes,att_df_glhymps])

print(debug_message)


!!! CHECK DEBUGGING STATUS: 
- Testing 1 file 
- Testing 1 basin
Processing geospatial data into attributes for CAN_01AM001
 - processing rdrs
Running distributed case for RDRS data.
Running subbasin 0
Running subbasin 1
Running subbasin 2
Running subbasin 3
Running subbasin 4
Running subbasin 5
Running subbasin 6
Running subbasin 7
Running subbasin 8
Running subbasin 9
Running subbasin 10
 - processing worldclim


/globalhome/wmk934/HPC/camels_spat/camels-spat-env-jupyter/lib/python3.10/site-packages/rasterstats/io.py:328: NodataWarning: Setting nodata to -999; specify nodata explicitly
  warnings.warn(


 - processing lai
 - processing forest_height
 - processing glclu2019
 - processing modis_land
 - processing lgrip30
 - processing merit
 - processing hydrolakes
 - processing pelletier
 - processing soilgrids
 - processing glhymps


/opt/software/custom/software/2020/avx2/MPI/gcc9/openmpi4/geo-stack/2022a/lib/python3.10/site-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,



!!! CHECK DEBUGGING STATUS: 
- Testing 1 file 
- Testing 1 basin


In [9]:
att_df

72047261.0  \
Category Attribute             Unit  Source                        
Climate  num_years_rdrs        years RDRS                     37   
         PR0_SFC_mean          mm    RDRS     1178.1281495076287   
         PR0_SFC_std           mm    RDRS      149.1360432992738   
         pet_mean              mm    RDRS      1135.972213843205   
         pet_std               mm    RDRS     51.072779601267264   
...                                                          ...   
Geology  porosity_std          -     GLHYMPS                 0.0   
         log_permeability_min  m^2   GLHYMPS               -15.2   
         log_permeability_mean m^2   GLHYMPS               -15.2   
         log_permeability_max  m^2   GLHYMPS               -15.2   
         log_permeability_std  m^2   GLHYMPS                 0.0   

                                                      72047339.0  \
Category Attribute             Unit  Source                        
Climate  num_years_rdrs        years RDRS                     37   
         PR0_SFC_mean          mm    RDRS      1190.216598959602   
         PR0_SFC_std           mm    RDRS     152.52595093657615   
         pet_mean              mm    RDRS      1126.404816187396   
         pet_std               mm    RDRS         50.55141125513   
...                                                          ...   
Geology  porosity_std          -     GLHYMPS                 0.0   
         log_permeability_min  m^2   GLHYMPS               -15.2   
         log_permeability_mean m^2   GLHYMPS               -15.2   
         log_permeability_max  m^2   GLHYMPS               -15.2   
         log_permeability_std  m^2   GLHYMPS                 0.0   

                                                      72047482.0  \
Category Attribute             Unit  Source                        
Climate  num_years_rdrs        years RDRS                     37   
         PR0_SFC_mean          mm    RDRS     1195.2213394437126   
         PR0_SFC_std           mm    RDRS      156.2346445042371   
         pet_mean              mm    RDRS      1120.884753996585   
         pet_std               mm    RDRS     51.576821505826686   
...                                                          ...   
Geology  porosity_std          -     GLHYMPS                 0.0   
         log_permeability_min  m^2   GLHYMPS               -15.2   
         log_permeability_mean m^2   GLHYMPS               -15.2   
         log_permeability_max  m^2   GLHYMPS               -15.2   
         log_permeability_std  m^2   GLHYMPS                 0.0   

                                                      72047855.0  \
Category Attribute             Unit  Source                        
Climate  num_years_rdrs        years RDRS                     37   
         PR0_SFC_mean          mm    RDRS     1187.1226192793001   
         PR0_SFC_std           mm    RDRS     155.78782205101646   
         pet_mean              mm    RDRS     1120.0700821635012   
         pet_std               mm    RDRS      51.85721375592091   
...                                                          ...   
Geology  porosity_std          -     GLHYMPS                 0.0   
         log_permeability_min  m^2   GLHYMPS               -15.2   
         log_permeability_mean m^2   GLHYMPS               -15.2   
         log_permeability_max  m^2   GLHYMPS               -15.2   
         log_permeability_std  m^2   GLHYMPS                 0.0   

                                                      72048203.0  \
Category Attribute             Unit  Source                        
Climate  num_years_rdrs        years RDRS                     37   
         PR0_SFC_mean          mm    RDRS     1211.7013558289948   
         PR0_SFC_std           mm    RDRS     164.28329058163752   
         pet_mean              mm    RDRS     1087.9457710392621   
         pet_std               mm    RDRS     47.389855566281405   
...                                   

In [10]:
# Save to file
att_file = f'attributes_{basin_id}.csv'
att_df.to_csv(att_path/att_file)